In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

# Load the data
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

data_train

,season,price_AUS,price_CHF,price_CZE,price_GER,price_ESP,price_FRA,price_UK,price_ITA,price_POL,price_SVK
0,spring,NaN,9.644028,-1.686248,-1.748076,-3.666005,NaN,-1.822720,-3.931031,NaN,-3.238197
1,summer,NaN,7.246061,-2.132377,-2.054363,-3.295697,-4.104759,-1.826021,NaN,NaN,-3.212894
2,autumn,-2.101937,7.620085,-1.910282,NaN,-3.388777,NaN,-2.034409,-4.073850,NaN,-3.114061
3,winter,-2.098475,8.411894,-1.903834,NaN,-3.588235,NaN,-2.214720,-4.018620,-2.330803,NaN
4,spring,-1.969687,8.926884,-1.697257,-1.331049,NaN,-3.911096,-2.388092,-4.093946,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
895,winter,-1.044730,NaN,0.190134,0.253153,-3.443941,-1.967611,NaN,-1.838593,3.235645,NaN
896,spring,-1.061639,-1.734754,0.281646,NaN,-3.466753,-1.929701,NaN,-1.508756,3.187263,NaN
897,summer,-0.971157,NaN,0.245279,0.558474,NaN,NaN,-0.843659,-1.499361,3.110638,2.230253
898,autumn,NaN,NaN,0.299911,0.788152,-3.339650,NaN,-0.865169,-1.547716,3.105417,1.989140


In [2]:
from sklearn.ensemble import RandomForestRegressor

train_gender_mapping = {label: idx for idx, label in enumerate(set(data_train['season']))}
data_train = pd.get_dummies(data_train)

# Identify columns with missing values
missing_columns = data_train.columns[data_train.isna().any()].tolist()

# Impute missing values using RandomForestRegressor
for column in missing_columns:
    # Split data into sets with and without missing values
    missing_data = data_train[data_train[column].isna()]
    non_missing_data = data_train.dropna(subset=[column])
    
    # Prepare features (X) and target (y) for training
    X_train = non_missing_data.drop(missing_columns, axis=1)
    y_train = non_missing_data[column]
    
    # Train the RandomForestRegressor model
    rf = RandomForestRegressor(n_estimators=100, random_state=0)
    rf.fit(X_train, y_train)
    
    # Prepare features (X) for prediction
    X_missing = missing_data.drop(missing_columns, axis=1)
    
    # Predict missing values and update the dataset
    data_train.loc[missing_data.index, column] = rf.predict(X_missing)

data_train.describe()

,price_AUS,price_CHF,price_CZE,price_GER,price_ESP,price_FRA,price_UK,price_ITA,price_POL,price_SVK,season_autumn,season_spring,season_summer,season_winter
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,-0.664068,4.540643,-0.222123,-0.421332,-4.739479,-2.913491,-1.517745,-2.685926,-0.297502,-0.644141,0.250000,0.250000,0.250000,0.250000
std,0.856089,2.787828,0.999232,0.967013,0.993247,0.926979,1.331160,0.965693,1.690685,1.736207,0.433253,0.433253,0.433253,0.433253
min,-2.362783,-3.736940,-2.147384,-2.238546,-6.891480,-4.428215,-3.940520,-4.330588,-2.912800,-3.499810,0.000000,0.000000,0.000000,0.000000
25%,-1.130879,3.099984,-0.944784,-1.244850,-5.355475,-3.757040,-2.174116,-3.364211,-1.868955,-2.214581,0.000000,0.000000,0.000000,0.000000
50%,-0.652056,4.510274,-0.175067,-0.410124,-4.733637,-2.937618,-1.589110,-2.679521,-0.349505,-0.655941,0.000000,0.000000,0.000000,0.000000
75%,-0.265247,6.870212,0.546098,0.260353,-3.905890,-2.520053,-1.474406,-2.229640,0.654476,0.643450,0.250000,0.250000,0.250000,0.250000
max,1.316798,9.933313,1.710173,1.659539,-2.858268,0.000619,1.624482,0.943226,3.421927,2.527192,1.000000,1.000000,1.000000,1.000000


In [ ]:
y = data_train['price_CHF']
X = data_train.drop(['price_CHF'], axis=1)